<a href="https://colab.research.google.com/github/Flabert/Calculator/blob/main/Random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 Import libraries
 import numpy as np
 import matplotlib.pyplot as plt
 import seaborn as sns
 from sklearn.datasets import load_iris
 from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
 from sklearn.tree import DecisionTreeClassifier, plot_tree
 from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
 import joblib

#load the iris dataset
iris = load_iris()
x = iris.data
y = iris.target

#Basic dataset overview
print (f"Dataset shape: {x.shape}")
print (f"Classes: {iris.target_names}")
print (f"Feature names: {iris.feature_names}")

"""
Visualize pairplot to see the relationship between features
Pairplot shows the relationship between different features, coloured by species,
giving a visula understanding of the dataset structure
"""
sns.pairplot(sns.load_dataset("iris"), hue="species")
plt.show()












